# Go Further with PostGIS and Plotly
Assumptions: <br>
- PostgreSQL is installed locally

## Let's Setup

Download and unzip the Plotly Database Connector app

In [ ]:
!wget https://github.com/plotly/plotly-database-connector/releases/download/v0.0.7-alpha/Plotly.Database.Connector-Mac.zip

In [ ]:
!unzip Plotly.Database.Connector-Mac.zip -d ./

Download and unzip the data required for this example

In [ ]:
!wget https://github.com/plotly/plotly-database-connector/tree/master/examples/postgis/dc/dc_census.zip

In [ ]:
!unzip dc_census.zip -d ./

Start your postgres server with a command similar to this (depending on where you installed PostgreSQL)<BR>
$ postgres -D /usr/local/pgsql/data

In [ ]:
# Create a new postgreSQL database called 'dc_census_tracts'
!createdb dc_census_tracts

In [ ]:
# add postgis language to the postgis database
!createlang plpgsql dc_census_tracts
# will get the following if it is there already :
# $ language "plpgsql" is already installed in database "postgis"

In [ ]:
# install the postgis extensions to the postgis database
!psql -d dc_census_tracts -c "CREATE EXTENSION postgis;"
!psql -d dc_census_tracts -c "CREATE EXTENSION postgis_topology;"
# will get the following if they are already installed:
# ERROR:  extension "postgis" already exists
# ERROR:  extension "postgis_topology" already exists

In [ ]:
!cd dc_census && ls

In [ ]:
# Import shapefile
!shp2pgsql -c -D -s 4269 -I dc_census/tl_2010_11001_tract10.shp dc_census_tracts | psql -d dc_census_tracts

Create a table in the database by entering the pgsql prompt: <br>
$ psql dc_census_tracts
and entering the following SQL query into `dc_census_tracts=#` prompt
```
CREATE TABLE dc_census_data (GEOID varchar(11), SUMLEV varchar(3), STATE varchar(2), COUNTY varchar(3), CBSA varchar(5), CSA varchar(3), NECTA integer, CNECTA integer, NAME varchar(30), POP100 integer, HU100 integer, POP1002000 integer, HU1002000 integer, P001001 integer, P0010012000 integer);
```

In [ ]:
!cat dc_census/all_140_in_11.P1.csv | psql -d dc_census_tracts -c 'COPY dc_census_data FROM STDIN WITH CSV HEADER'

## Let's Connect To Our Database

Start up the Plotly Database Application and connect to the `dc_census_tracts` database.

In [ ]:
!open ./Plotly\ Database\ Connector-darwin-x64/Plotly\ Database\ Connector.app

Follow the instructions until you are connected and can view the desired tables.

In [ ]:
import pandas as pd

In [1]:
import requests

Let's make sure the app is connected by using it's API.

In [ ]:
auth = requests.get('http://localhost:5000/v1/authenticate')

In [ ]:
auth.json()

The API permits us to switch databases if we have to, since we want to use `dc_census_data` selecting a database here is optional but here is how that would work:

In [ ]:
connectDatabase = requests.get('http://localhost:5000/v1/selectdatabase?database=dc_census_tracts')

In [ ]:
connectDatabase.json()

We just created two tables in that databse: `dc_census_tacts` and `dc_census_data`. Let's make sure they are there by retreiving the list of tables from our database.

In [ ]:
tables = requests.get('http://localhost:5000/v1/tables')

In [ ]:
tables.json()

Looks like both `u'dc_census_tracts` and `u'dc_census_data` are there!

## Let's Explore Our Data

In [ ]:
response = requests.get('http://localhost:5000/v1/preview?tables=dc_census_tracts,dc_census_data')

The `response` received has a `previews` object that contains the first five rows of each table specified as in the above `tables` parameter of the request. Let's get into the table `dc_census_tracts` and see the geojson of the fifth row (index 4) row only. To get the geojson object we enter in to the `geom` key.

In [ ]:
response.json()['previews'][0]['dc_census_tracts']['raw'][0]['geom']

Looks like the data is a collection of complex Polygons.

Let's look at the other table, `dc_census_data`, whose preview is also in our response object.

In [ ]:
df = pd.DataFrame(response.json()['previews'][1]['dc_census_data']['rows'])
df.columns = response.json()['previews'][1]['dc_census_data']['columnnames']

In [ ]:
df

Looks like it has population data for each county

## Let's Extract Our Data

We can define right from the start how much data exactly we want to visualize. Use an integer as `LIMIT` value or set it simply to be null i.e. `LIMIT = ''`

In [17]:
LIMIT = '' #'LIMIT 10'

Let's combine both tables and do some analysis.

Right before, let's add a column that will have the centroid of each county. <br>
Run these commands in the psql prompt.

`ALTER TABLE "dc_census_tracts" ADD centroid_geom geometry;` <br>
`UPDATE "dc_census_tracts" SET centroid_geom = ST_Centroid(geom);`

In [5]:
query = 'SELECT * from dc_census_tracts JOIN dc_census_data on dc_census_tracts.geoid10 = dc_census_data.geoid ' + LIMIT

With the connector API we can send our own queries as well

In [6]:
queryResponse = requests.get('http://localhost:5000/v1/query?statement=' + query)

In [ ]:
# queryResponse.json()

Looks like we have the data we need, let's create a geometries object that we can use when drawing shapes using plotly! These geometry objects are inside our data under the `geom` key.

## Let's Process Our Data

We only need the raw response from PostGIS, let's put that into a local variable and go from there.

In [7]:
locations = queryResponse.json()['raw']

#### 1 Sectors of Counties

In [8]:
geometries = [location['geom'] for location in locations]

In [9]:
geojsons = [{
    "type": "FeatureCollection",
    "features": [{
        "type": "Feature",
        "properties": {},
        "geometry": {
            "type": "GeometryCollection",
            "geometries": [geometry]
        }
    }]
} for geometry in geometries]

#### 2 Centroids of Counties

In [10]:
centroids = [location['centroid_geom'] for location in locations]

In [ ]:
# centroids

#### 3 Populations of Counties

In [11]:
populations = [location['pop100'] for location in locations]

In [ ]:
# populations

#### 4 A map of DC has to have the White House on it...

In [12]:
USA_HQ = dict(
            lon='-77.0365',
            lat='38.8977'
        )

## Let's Make a Plot!

In [13]:
import plotly.plotly as py
import plotly.tools as tls
from plotly.graph_objs import *

In [14]:
mapbox_access_token = 'pk.eyJ1IjoiY2hyaWRkeXAiLCJhIjoiRy1GV1FoNCJ9.yUPu7qwD_Eqf_gKNzDrrCQ'

In [16]:
data = Data([
    Scattermapbox(
        name='USA HQ',
        lat=['38.8977'],
        lon=['-77.0365'],
        mode='markers',
        marker=Marker(
            size=10
        ),
        text=['Barack Obama lives in this house']
    ),
    Scattermapbox(
        name='County Populations',
        lat=[str(centroid['coordinates'][1]) for centroid in centroids],
        lon=[str(centroid['coordinates'][0]) for centroid in centroids],
        mode='markers',
        marker=Marker(
            size=10
        ),
        text=[str(population) + ' people live in this county' for population in populations]
    )
])

In [17]:
layout = Layout(
    autosize=True,
    hovermode='closest',
    mapbox=dict(
        accesstoken=mapbox_access_token,
        center=dict(
            lat=38.8977,
            lon=-77.0365
        ),
        pitch=0,
        zoom=12,
        layers=[
            {
                'sourcetype':'geojson',
                'source': geojson,
                'type': 'fill',
                'color': 'rgba(30, 30, 30, 0.2)'            
            } for geojson in geojsons
        ]
    )
)

In [18]:
fig = dict(data=data, layout=layout)

In [19]:
tls.set_credentials_file(username='', api_key='')

In [20]:
py.iplot(fig, filename='dc_census')

## Exploring Possible Queries

In [2]:
import requests

#### 1 - Modify the Data Type upon Extracting It

Simply add a `ST_AsTypeWanted` function in front of the geom extracted.

The raw `geom` data here is stored as a SRID = 4269

Looks like something like this: `0101000020AD100000BAA0BE654E5F52C0AFB14B546FB54640`

In [12]:
query = 'SELECT ST_SRID(geom) FROM dc_census_tracts LIMIT 1;'
queryResponse = requests.get('http://localhost:5000/v1/query?statement=' + query)
queryResponse.json()['raw']

[{u'st_srid': 4269}]

When 'extracted' with no specification:

In [13]:
query = 'SELECT geom FROM dc_census_tracts LIMIT 1;'
queryResponse = requests.get('http://localhost:5000/v1/query?statement=' + query)
queryResponse.json()['raw']

[{u'geom': {u'coordinates': [[[[-77.027429, 38.951868],
      [-77.02726799999999, 38.951867],
      [-77.02622099999999, 38.951898],
      [-77.025551, 38.951917],
      [-77.025306, 38.951924],
      [-77.02492099999999, 38.951932],
      [-77.024763, 38.951938999999996],
      [-77.024621, 38.951944],
      [-77.024003, 38.951962],
      [-77.02393, 38.951963],
      [-77.02382899999999, 38.951966],
      [-77.023727, 38.951968],
      [-77.02296799999999, 38.951989],
      [-77.022204, 38.952014],
      [-77.022104, 38.952016],
      [-77.021845, 38.952023],
      [-77.021562, 38.952033],
      [-77.021177, 38.952041],
      [-77.020766, 38.952054],
      [-77.019885, 38.952076999999996],
      [-77.01968099999999, 38.952083],
      [-77.019632, 38.951021],
      [-77.019607, 38.950466999999996],
      [-77.019603, 38.950402],
      [-77.01959599999999, 38.950249],
      [-77.019584, 38.950026],
      [-77.019581, 38.949951999999996],
      [-77.019579, 38.949920999999996],
      [

As GeoJSON:

http://postgis.net/docs/ST_AsGeoJSON.html

In [14]:
query = 'SELECT ST_AsGeoJSON(geom) FROM dc_census_tracts LIMIT 1;'
queryResponse = requests.get('http://localhost:5000/v1/query?statement=' + query)
queryResponse.json()['raw']

[{u'st_asgeojson': u'{"type":"MultiPolygon","coordinates":[[[[-77.027429,38.951868],[-77.027268,38.951867],[-77.026221,38.951898],[-77.025551,38.951917],[-77.025306,38.951924],[-77.024921,38.951932],[-77.024763,38.951939],[-77.024621,38.951944],[-77.024003,38.951962],[-77.02393,38.951963],[-77.023829,38.951966],[-77.023727,38.951968],[-77.022968,38.951989],[-77.022204,38.952014],[-77.022104,38.952016],[-77.021845,38.952023],[-77.021562,38.952033],[-77.021177,38.952041],[-77.020766,38.952054],[-77.019885,38.952077],[-77.019681,38.952083],[-77.019632,38.951021],[-77.019607,38.950467],[-77.019603,38.950402],[-77.019596,38.950249],[-77.019584,38.950026],[-77.019581,38.949952],[-77.019579,38.949921],[-77.019566,38.949637],[-77.019553,38.949401],[-77.019546,38.949211],[-77.019544,38.949135],[-77.019528,38.948812],[-77.019513,38.94847],[-77.019495,38.948149],[-77.019469,38.94764],[-77.019459,38.947402],[-77.019456,38.947308],[-77.019413,38.946335],[-77.020235,38.946312],[-77.020764,38.946296]

As Text:

In [19]:
query = 'SELECT ST_AsText(geom) FROM dc_census_tracts LIMIT 1;'
queryResponse = requests.get('http://localhost:5000/v1/query?statement=' + query)
queryResponse.json()['raw']

[{u'st_astext': u'MULTIPOLYGON(((-77.027429 38.951868,-77.027268 38.951867,-77.026221 38.951898,-77.025551 38.951917,-77.025306 38.951924,-77.024921 38.951932,-77.024763 38.951939,-77.024621 38.951944,-77.024003 38.951962,-77.02393 38.951963,-77.023829 38.951966,-77.023727 38.951968,-77.022968 38.951989,-77.022204 38.952014,-77.022104 38.952016,-77.021845 38.952023,-77.021562 38.952033,-77.021177 38.952041,-77.020766 38.952054,-77.019885 38.952077,-77.019681 38.952083,-77.019632 38.951021,-77.019607 38.950467,-77.019603 38.950402,-77.019596 38.950249,-77.019584 38.950026,-77.019581 38.949952,-77.019579 38.949921,-77.019566 38.949637,-77.019553 38.949401,-77.019546 38.949211,-77.019544 38.949135,-77.019528 38.948812,-77.019513 38.94847,-77.019495 38.948149,-77.019469 38.94764,-77.019459 38.947402,-77.019456 38.947308,-77.019413 38.946335,-77.020235 38.946312,-77.020764 38.946296,-77.021099 38.946285,-77.021224 38.94628,-77.021412 38.946273,-77.021832 38.946261,-77.022526 38.946238,-77.0

Note that the `type` parameter acts as a 'constructor' and receives the list of points. The whole Object is a string as expected. 

Can also use these other types when extracting data:
- ST_AsBinary — Return the Well-Known Binary (WKB) representation of the geometry/geography without SRID meta data.
- ST_AsEWKB — Return the Well-Known Binary (WKB) representation of the geometry with SRID meta data.
- ST_AsEWKT — Return the Well-Known Text (WKT) representation of the geometry with SRID meta data.
- ST_AsGeoJSON — Return the geometry as a GeoJSON element.
- ST_AsGML — Return the geometry as a GML version 2 or 3 element.
- ST_AsHEXEWKB — Returns a Geometry in HEXEWKB format (as text) using either little-endian (NDR) or big-endian (XDR) encoding.
- ST_AsKML — Return the geometry as a KML element. Several variants. Default version=2, default precision=15
- ST_AsSVG — Returns a Geometry in SVG path data given a geometry or geography object.
- ST_GeoHash — Return a GeoHash representation (geohash.org) of the geometry.
- ST_AsText — Return the Well-Known Text (WKT) representation of the geometry/geography without SRID metadata.


#### 2 -  Advanced Queries

#### Distance From a Point Provided as Text

NOTE: SRID by defaults when read from text is 0, to make computations using that geometry, set the SRID like so <br>
`ST_GeomFromText('SRID=4269;POINT(-73 45)')`

In [23]:
query = "\
SELECT name, ST_Distance(geom, ST_GeomFromText('SRID=4269;POINT(-73 45)')) \
FROM montreal_places LIMIT 5;\
"
queryResponse = requests.get('http://localhost:5000/v1/query?statement=' + query)
queryResponse.json()['raw']

[{u'name': u'La Prairie', u'st_distance': 0.643078811033295},
 {u'name': u'Pointe-Claire', u'st_distance': 0.927025844569616},
 {u'name': u'Dorval', u'st_distance': 0.87316278917508},
 {u'name': u'Beaconsfield', u'st_distance': 0.968098237835403},
 {u'name': u'Kirkland', u'st_distance': 0.976176229274204}]

Here we received a number instead of an object. Can we catch and parse this automatically or leave it to the user? <br> 
`AsGeoJSON()` isn't useful here.

#### Filter by Distance From a Point

In [27]:
query = "\
SELECT name, geom from montreal_places \
WHERE ST_Distance(geom, ST_GeomFromText('SRID=4269;POINT(-73 45)')) < 0.6;\
"
queryResponse = requests.get('http://localhost:5000/v1/query?statement=' + query)
queryResponse.json()['raw']

[{u'geom': {u'coordinates': [-73.28765, 45.52624], u'type': u'Point'},
  u'name': u'Saint-Basile-le-Grand'},
 {u'geom': {u'coordinates': [-73.34449, 45.31491], u'type': u'Point'},
  u'name': u"L'Acadie"},
 {u'geom': {u'coordinates': [-73.30368, 45.36214], u'type': u'Point'},
  u'name': u'Saint-Luc'},
 {u'geom': {u'coordinates': [-73.29031, 45.4474], u'type': u'Point'},
  u'name': u'Chambly'},
 {u'geom': {u'coordinates': [-73.47419, 45.35264], u'type': u'Point'},
  u'name': u'Saint-Philippe-de-La Prairie'},
 {u'geom': {u'coordinates': [-73.30431, 45.42037], u'type': u'Point'},
  u'name': u'Mont\xe9r\xe9gie'},
 {u'geom': {u'coordinates': [-73.29174, 45.47089], u'type': u'Point'},
  u'name': u'\xcele aux Li\xe8vres'},
 {u'geom': {u'coordinates': [-73.29001, 45.46669], u'type': u'Point'},
  u'name': u'\xcele Demers'},
 {u'geom': {u'coordinates': [-73.35549, 45.43795], u'type': u'Point'},
  u'name': u'Carignan'}]

#### Find Area of Features (Polygons)

In [34]:
query = "\
SELECT ST_Area(geom)*10000\
FROM dc_census_tracts \
LIMIT 5;\
"
queryResponse = requests.get('http://localhost:5000/v1/query?statement=' + query)
queryResponse.json()['rows']

[[0.431404620000005],
 [0.722721385000264],
 [0.428706184999896],
 [2.09379702500001],
 [0.578057435000104]]

#### Find box that englobes a complicated shape (polygon)

In [ ]:
query = "\
SELECT ST_Area(geom)*10000\
FROM dc_census_tracts \
LIMIT 5;\
"
queryResponse = requests.get('http://localhost:5000/v1/query?statement=' + query)
queryResponse.json()['rows']

#### 4 - Geography

Creating a table with a `GEOGRAPHY` type

```
CREATE TABLE global_points ( 
    id SERIAL PRIMARY KEY,
    name VARCHAR(64),
    location GEOGRAPHY(POINT,4326)
  );
``` 

```
INSERT INTO global_points (name, location) VALUES ('Singapore', ST_GeographyFromText('SRID=4326;POINT(103.81 1.3521)') );
INSERT INTO global_points (name, location) VALUES ('Montreal', ST_GeographyFromText('SRID=4326;POINT(-73.56 45.5)') );
INSERT INTO global_points (name, location) VALUES ('London', ST_GeographyFromText('SRID=4326;POINT(0 49)') );
```

You can see the power of GEOGRAPHY in action by calculating the how close a plane flying from Seattle to London (LINESTRING(-122.33 47.606, 0.0 51.5)) comes to Montreal (POINT(-73.29031 45.4474))

In [37]:
query = "\
SELECT * FROM global_points;\
"
queryResponse = requests.get('http://localhost:5000/v1/query?statement=' + query)
queryResponse.json()['rows']

[[3, u'London', {u'coordinates': [0, 49], u'type': u'Point'}],
 [5, u'Montreal', {u'coordinates': [-73.56, 45.5], u'type': u'Point'}],
 [6, u'Singapore', {u'coordinates': [103.81, 1.3521], u'type': u'Point'}]]

Same default format (GeoJSON) as geometry from above examples.

Distance calculations using geography points, their SRID is 4326.

In [43]:
query = "\
SELECT name, ST_Distance(location, ST_GeogFromText('SRID=4326;POINT(-73 45)'))/1000 \
FROM global_points;\
"
queryResponse = requests.get('http://localhost:5000/v1/query?statement=' + query)
queryResponse.json()['rows']

[[u'London', 5351.36474657502],
 [u'Montreal', 70.85493040627],
 [u'Singapore', 14859.8721747657]]